In [2]:
from haystack.document_stores import FAISSDocumentStore, InMemoryDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.pipelines import DocumentSearchPipeline

In [ ]:
# to make all haystack code works Python 3.10 was used 
# (there was some errors with faiss installation with Python 3.12)
# to avoid package building errors 'numpy<2' was installed

In [3]:
from datasets import load_dataset

ds_qa = load_dataset("clarin-knext/fiqa-pl-qrels")
ds_corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")
ds_queries = load_dataset("clarin-knext/fiqa-pl", "queries")

In [4]:
print(ds_corpus)

DatasetDict({
    corpus: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 57638
    })
})


In [5]:
ds_corpus['corpus'][:3]

{'_id': ['3', '31', '56'],
 'title': ['', '', ''],
 'text': ['Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.',
  'Tak więc nic nie zapobiega fałszywym ocenom poza dodatkową kontrolą ze strony rynku/inwestorów, ale istnieją pewne nowsze kontrole, które uniemożliwiają instytucjom korzystanie z nich. W ramach DFA banki nie mogą już polegać wyłącznie na ratingach kredytowych jako należytej staranności przy zakupie instrumentu finansowego, więc to jest plus. Intencją jest to, że jeśli instytucje finansowe wykonują swoją własną pracę, to *być może* dojdą do wniosku, że określony CDO

In [6]:
n = 5
for sample in ds_corpus['corpus']:
    n -= 1
    if n < 0:
        break
    print(f"{sample['_id']} : {sample['text']}")

3 : Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.
31 : Tak więc nic nie zapobiega fałszywym ocenom poza dodatkową kontrolą ze strony rynku/inwestorów, ale istnieją pewne nowsze kontrole, które uniemożliwiają instytucjom korzystanie z nich. W ramach DFA banki nie mogą już polegać wyłącznie na ratingach kredytowych jako należytej staranności przy zakupie instrumentu finansowego, więc to jest plus. Intencją jest to, że jeśli instytucje finansowe wykonują swoją własną pracę, to *być może* dojdą do wniosku, że określony CDO jest śmieciem, czy nie. Edycja: wprowadzenie
56 : Nigdy n

In [7]:
documents = [{"content": sample['text'], "meta": {"_id": sample['_id']}} for sample in ds_corpus['corpus']]

In [8]:
documents[:3]

[{'content': 'Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.',
  'meta': {'_id': '3'}},
 {'content': 'Tak więc nic nie zapobiega fałszywym ocenom poza dodatkową kontrolą ze strony rynku/inwestorów, ale istnieją pewne nowsze kontrole, które uniemożliwiają instytucjom korzystanie z nich. W ramach DFA banki nie mogą już polegać wyłącznie na ratingach kredytowych jako należytej staranności przy zakupie instrumentu finansowego, więc to jest plus. Intencją jest to, że jeśli instytucje finansowe wykonują swoją własną pracę, to *być może* dojdą do wniosku, że określony CDO jest śmieci

In [16]:
document_store = FAISSDocumentStore(
        faiss_index_factory_str="Flat",
        embedding_dim=768, # according to https://huggingface.co/intfloat/multilingual-e5-base
        similarity="cosine"
    )

In [17]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="intfloat/multilingual-e5-base",
    model_format="sentence_transformers",
    use_gpu=True
)

In [18]:
document_store.write_documents(documents)

Writing Documents: 60000it [00:52, 1143.19it/s]                                                                                    


In [ ]:
document_store.update_embeddings(retriever)

In [20]:
search_pipeline = DocumentSearchPipeline(retriever)

In [21]:
sample_question = ds_queries['queries'][0]['text']
print(f"{sample_question = }\n")

results = search_pipeline.run(query=sample_question, params={"Retriever": {"top_k": 3}})
results

sample_question = 'Co jest uważane za wydatek służbowy w podróży służbowej?'



Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 28.77it/s]


{'documents': [<Document: {'content': '„To zależy od tego, jaki jest „„prawdziwy” powód wyjazdu. Jeśli zdecydujesz się odliczyć wyjazd jako wydatek służbowy, to podczas audytu zostaniesz zapytany, dlaczego musiałeś tam pojechać. Jeśli nic nie zostało osiągnięte przez podróż (tj. pracowałeś w hotelu, nie spotkałeś się z żadnymi klientami, nie odwiedzałeś żadnych targów itp.), więc wydatek raczej nie będzie dozwolony. Tak, w podróży służbowej możesz zwiedzać, jeśli chcesz (choć możesz t odliczyć wszelkie wydatki związane ze zwiedzaniem, takie jak wstęp do atrakcji turystycznej), ale jeśli pracujesz tylko na wakacjach, sama podróż nie podlega odliczeniu, ponieważ podróżowanie nie przynosiło żadnych korzyści biznesowych”.', 'content_type': 'text', 'score': 0.9389107525348663, 'meta': {'_id': '531578', 'vector_id': '22558'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '6e254abf22a5f68f736dd471f7f30120'}>,
  <Document: {'content': 'Jedzenie prawie nigdy nie jest uzasadnionym wydatk

In [ ]:
# results seems to be reasonable

In [22]:
results['documents'][0].meta['_id']

'531578'

In [23]:
results['documents'][0].content

'„To zależy od tego, jaki jest „„prawdziwy” powód wyjazdu. Jeśli zdecydujesz się odliczyć wyjazd jako wydatek służbowy, to podczas audytu zostaniesz zapytany, dlaczego musiałeś tam pojechać. Jeśli nic nie zostało osiągnięte przez podróż (tj. pracowałeś w hotelu, nie spotkałeś się z żadnymi klientami, nie odwiedzałeś żadnych targów itp.), więc wydatek raczej nie będzie dozwolony. Tak, w podróży służbowej możesz zwiedzać, jeśli chcesz (choć możesz t odliczyć wszelkie wydatki związane ze zwiedzaniem, takie jak wstęp do atrakcji turystycznej), ale jeśli pracujesz tylko na wakacjach, sama podróż nie podlega odliczeniu, ponieważ podróżowanie nie przynosiło żadnych korzyści biznesowych”.'

In [9]:
simplified_dict = {q_id: [] for q_id in set(ds_qa["test"]['query-id'])}
simplified_query_test_only = {}

for i in range(len(ds_qa["test"])):
    qa = ds_qa["test"][i]
    simplified_dict[qa["query-id"]].append(qa["corpus-id"])

for i in range(len(ds_queries['queries'])):
    if int(q_id := ds_queries['queries'][i]['_id']) in simplified_dict:
        simplified_query_test_only[int(q_id)] = ds_queries['queries'][i]['text']

In [10]:
list(simplified_query_test_only.values())[:5]

['Gdzie powinienem zaparkować mój fundusz na deszczowy dzień / awaryjny?',
 'Względy podatkowe związane ze sprzedażą rodzinie nieruchomości poniżej szacunkowej wartości?',
 'Czy Delta może być wykorzystana do obliczenia premii opcyjnej przy określonym celu?',
 'Podstawowa strategia handlu algorytmicznego',
 'Co oznacza dla firmy wysoka marża operacyjna, ale niewielkie, ale dodatnie ROE?']

In [36]:
queries = list(simplified_query_test_only.values())[:5]

multi_results = search_pipeline.run_batch(queries=queries, params={"Retriever": {"top_k": 5}})

for query, result in zip(queries, multi_results['documents']):
    print(f"\n\nQuery: {query}\n")
    for text in result:
        print(f"{text.meta['_id']} : {text.content} (Score: {text.score})")

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.62it/s]




Query: Gdzie powinienem zaparkować mój fundusz na deszczowy dzień / awaryjny?

376148 : Obligacje niekoniecznie są bezpieczniejsze niż na giełdzie. Ostatecznie nie ma czegoś takiego jak fundusz inwestycyjny niskiego ryzyka. Chcesz czegoś, co pozwoli Ci stosunkowo szybko uzyskać pieniądze. Innymi słowy, płyty CD (ponieważ możesz wybrać określony czas na zawiązanie pieniędzy), konto rynku pieniężnego lub po prostu zwykłe stare konto oszczędnościowe. Zasadniczo chcesz dopasować inflację i mieć łatwy dostęp do pieniędzy. Wszelkie inne zwroty poza tym są sosem, ale nie przejmuj się tym zbytnio. Zobacz też: Gdzie mogę zaparkować mój fundusz na deszczowy dzień / awaryjny? Konta oszczędnościowe nie generują dużego zainteresowania. Gdzie powinienem zaparkować mój fundusz na deszczowy dzień / awaryjny? (Score: 0.9544209837913513)
406219 : Sugerowałbym lokalną kasę kredytową lub lokalny bank dla bezpieczeństwa i płynności. Płynność jest prawdopodobnie najważniejszą kwestią dla funduszu awaryjne

In [11]:
import numpy as np

def ndcg5(hits_ids, relevant_ids):
    ideal_dcg = 0
    dcg = 0
    
    for i in range(5):
        if hits_ids[i] in relevant_ids:   
            dcg += 1/np.log2(i+2)

    for i in range(len(relevant_ids)):
        ideal_dcg += 1/np.log2(i+2)

    return dcg/ideal_dcg


In [12]:
def calc_ndcg(queries_dct):
    ndcgs = []
    K = 5

    queries = list(simplified_query_test_only.values())
    queries_ids = list(simplified_query_test_only.keys())
    results = search_pipeline.run_batch(queries=queries, params={"Retriever": {"top_k": K}})
    for query_id, result in zip(queries_ids, results["documents"]):
        relevant_ids = simplified_dict[query_id]
        hit_ids = [int(text.meta['_id']) for text in result]
        ndcgs.append(ndcg5(hit_ids, relevant_ids))

    return np.mean(ndcgs)

In [50]:
len(list(simplified_query_test_only.keys()))

648

In [51]:
mean_ndcg = calc_ndcg(simplified_query_test_only)
print(f"ndcg with dense text representations (multilingual-e5-base model) - {mean_ndcg}")

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:05<00:00,  3.93it/s]


ndcg with dense text representations (multilingual-e5-base model) - 0.20606707793100607


## Podsumowanie

Uzyskane ndcg@5 na pozimie 0.206 jest najwyższym wynikiem w tej metryce, jaki udało się uzyskać. Najwyższy wynik NDCG@5 jaki udało się uzyskać z wykorzystaniem samego ElasticSearch to 0.182 - połączeie ES z dodatkowym transformerem do rerankingu dawało jeszcze gorszy wynik, na poziomie zaledwie 0.121. Chociaż NDCG@5 na poziomie około 20% trudno uznać za zadowalające, jest to wynik widocznie lepszy od innych testowanych rozwiązań. Kosztowene obliczeniowo i zajumjące więcej czasu było tylko początkowe obliczenie embedingów dla tekstów dodanych do bazy. Późniejsze uzyskanie odpowiedzi dla zbioru testowego korpusu fiqa (648 pozycji) nie zajęło wiele czasu - wbudowane w framework haystach narzędzia umożliwiły wykonanie wszystkich obliczeń w zaledwie 5 sekund (z wykorzystaniem CPU) - znacznie krócej niż w przypadku rerankingu, porównywanie z ES. 

Inne plusy zastosowania gęstych reprezentacji:
 - z wykorzystaniem frameworka Haystack stosunkowo łatwo przygotować działający pipeline - wystarczy kilkadziesiąt linijek kodu i podstawowa znajomość modelu
 - możliwość testowania różnych dostępnych modeli, metryk podobieństwa etc.
 - dość szybkie działanie (po policzeniu embedingów)
 - wsparcie dla GPU, które przyspiesza tworzenie osadzeń, nawet przy użyciu większych modeli (chociaż zadażały się problemy i framework z niewiadomych przyczyn używał tylko CPU)

Minusy dense representation:
 - konieczność tworzenia od nowa bazy przy zmianie modelu, metryki lub innego parametru
 - niezbyt dobre w przypadku korpusów, których zwartość jest nieustannie zmieniana - usuwanie i dodawanie nowych tekstów wymaga obliczania nowych embedingów
 - instalacja i konfiguracja Haystack dość problematyczna (problemy z wersjami wymaganych przez Haystack pakietów z PyPI, konieczność dobrania odpowiedniej wersji Pythona (z Pythonem 3.10 działa, na Pythonie 3.12 nie udało się zainstalować wszystkich komponentów frameworka, wspomniane problemy z poprawnym wykorzystaniem GPU)

In [ ]:
# Check how it will preform with bigger model

document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat", 
    embedding_dim=1024, # dim for bigger model
    similarity="dot_product" # metric changed to dot_product
)

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="intfloat/multilingual-e5-large",
    model_format="sentence_transformers",
    use_gpu=True
)

document_store.write_documents(documents)
document_store.update_embeddings(retriever)
search_pipeline = DocumentSearchPipeline(retriever)

In [14]:
mean_ndcg = calc_ndcg(simplified_query_test_only)
print(f"ndcg with dense text representations (multilingual-e5-large, dot_product similarity) - {mean_ndcg}")

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 12.20it/s]


ndcg with dense text representations (multilingual-e5-large, dot_product similarity) - 0.27399091592656594


## Komentarz do dodatkowego testu

Zmiana modelu na E5 Large oraz metryki podobieństwa na dot_product przyniosła wyraźną poprawę działania modelu - uzyskane NDCG@5 na poziomie 0.273 jest wyraźnie wyższe niż wyniki uzyskiwane przez inne rozwiązania wykorzystywane w czasie ćwiczeń z tym zagadnieniem i zbiorem danych.

Przy zastosowaniu większego modelu do uzyskania wyników udało się poprawnie wykorzystać GPU (konfiguracja środowiska i używany sprzęt bez zmian), co sprawiło, że wszystkie 648 próbek ze zbioru testowego zostało przetworzonych w około sekundę - wyraźnie szybciej niż w przypadku mniejszego modelu na CPU. Również przetworzenie korpusu trwało wyraźnie krócej (30 minut vs około 2,5 h dla E5 base na CPU)